In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
EPS = 1e-15
from torch.nn import Parameter
from torch_sparse import SparseTensor
from torch_geometric.utils.num_nodes import maybe_num_nodes
import pandas as pd
import numpy as np
import torch_cluster  
random_walk = torch.ops.torch_cluster.random_walk
device = 'cpu'
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
from torch_geometric.datasets import Planetoid
beta = 4.5
#gama = 0.5

def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    print("AP： ",average_precision_score(true_edges, predict_edges))
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    #predict_edges[predict_edges>gama] = 1
    #predict_edges[predict_edges<=gama] = 0
    
    #print("预测准确率： ",accuracy_score(true_edges, predict_edges))
    #print("预测精确率： ",precision_score(true_edges, predict_edges, average='macro'))
    #print("召回率： ",recall_score(true_edges, predict_edges, average='macro'))
    #print("F1 SCORE： ",f1_score(true_edges, predict_edges, average='macro'))
    
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, ne, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        NE = ne
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, cc):
        CC = cc
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=0.4, momentum=0.9)
        #self.loss_function = torch.nn.L1Loss(reduction='sum')
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, cc, edge_train, edge_test):
        CC = cc
        GP = g
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach()
    
####################################### 以最大化预测效果为目标的类型质心更新 ########################################
class ClusteringLayer(nn.Module):
    """给定节点嵌入，更新类簇质心"""
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, x):
        NE = x
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(ne, ini_cc, g, edge_train, edge_test, num_epochs=5):        
    NE = ne
    CC = ini_cc
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=0.4, momentum=0.9)
    #loss_function = torch.nn.L1Loss(reduction='sum')
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach()

In [2]:
def get_cora_graph(filename_adj, nodes_number):
    datasets = Planetoid('./data', filename_adj)
    edges = datasets[0].edge_index
    
    #print("总边数： ", edges.shape[1])
    
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(edges.shape[1]):
        graph_np[edges[0,i], edges[1,i]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # 只取左上角矩阵
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # 只取左上角矩阵
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*4,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [3]:
filename = "Cora"    # 节点索引从0开始, sep=None
nodes_number = 2708   
train_edge, test_edge, train_mask, test_mask = get_cora_graph(filename, nodes_number)

In [4]:
len(train_edge),len(test_edge)

(23750, 1056)

In [5]:
embedding_dim = 12
n_clusters = 7
alpha = 1

ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
ini_node_embeddings = ini_embedding.weight.detach()

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(ini_node_embeddings)
cluster_centers = kmeans.cluster_centers_
cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)

raw_nodes_embedding = ini_node_embeddings     # t=0时刻的节点嵌入
raw_cluster_centers = cluster_centers     # t=0时刻的类簇质心
for module_epoch in range(800):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
    raw_cluster_centers = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)

######################### 模块循环 ： 0 ##########################
AP：  0.6022014947400893
AUC SCORE:  0.5977674471992653
AP：  0.6049067943132154
AUC SCORE:  0.5958161157024793
######################### 模块循环 ： 1 ##########################
AP：  0.6069762816756079
AUC SCORE:  0.5949731691919192
AP：  0.6158626395541922
AUC SCORE:  0.5997008436639117
######################### 模块循环 ： 2 ##########################
AP：  0.621338100886579
AUC SCORE:  0.6022763142791552
AP：  0.6219924732818332
AUC SCORE:  0.6029434974747475
######################### 模块循环 ： 3 ##########################
AP：  0.6221388253855342
AUC SCORE:  0.603772095959596
AP：  0.6225031852095788
AUC SCORE:  0.6034994834710744
######################### 模块循环 ： 4 ##########################
AP：  0.6234517618641986
AUC SCORE:  0.6062004993112948
AP：  0.6237402578659327
AUC SCORE:  0.6064193067033976
######################### 模块循环 ： 5 ##########################
AP：  0.6241317619683784
AUC SCORE:  0.6082199839302113
AP：  0.6246006089140029
A

AP：  0.8056760349061027
AUC SCORE:  0.7486799816345271
######################### 模块循环 ： 48 ##########################
AP：  0.8070265978606513
AUC SCORE:  0.7503013085399449
AP：  0.8077822827458176
AUC SCORE:  0.7508537075298438
######################### 模块循环 ： 49 ##########################
AP：  0.8087719260667676
AUC SCORE:  0.7520123106060606
AP：  0.8099387022378357
AUC SCORE:  0.7531027605601469
######################### 模块循环 ： 50 ##########################
AP：  0.8114452429216491
AUC SCORE:  0.7547384354912765
AP：  0.811827064576987
AUC SCORE:  0.7551437672176309
######################### 模块循环 ： 51 ##########################
AP：  0.8127625347421357
AUC SCORE:  0.7564494375573921
AP：  0.8142037518985801
AUC SCORE:  0.7578304350321396
######################### 模块循环 ： 52 ##########################
AP：  0.8157795933938461
AUC SCORE:  0.7596275252525252
AP：  0.8164551073150201
AUC SCORE:  0.7604812327823691
######################### 模块循环 ： 53 ##########################
AP：  0.81773381886

AP：  0.8665442648459467
AUC SCORE:  0.8243155991735537
AP：  0.8668154609140087
AUC SCORE:  0.8248141930670341
######################### 模块循环 ： 96 ##########################
AP：  0.8671831445728931
AUC SCORE:  0.825366592056933
AP：  0.8673168955459448
AUC SCORE:  0.8255280073461891
######################### 模块循环 ： 97 ##########################
AP：  0.8679095338691374
AUC SCORE:  0.8264678030303031
AP：  0.8681413745345715
AUC SCORE:  0.8268157426538107
######################### 模块循环 ： 98 ##########################
AP：  0.8684479849622881
AUC SCORE:  0.8272139003673095
AP：  0.8686549262271556
AUC SCORE:  0.8275331439393939
######################### 模块循环 ： 99 ##########################
AP：  0.8689716255080936
AUC SCORE:  0.8280066287878789
AP：  0.8691917169973965
AUC SCORE:  0.8283438073921028
######################### 模块循环 ： 100 ##########################
AP：  0.8696132384308589
AUC SCORE:  0.8289643595041323
AP：  0.869674013596045
AUC SCORE:  0.8290576216712582
######################### 

AP：  0.8844356658656186
AUC SCORE:  0.8501743285123967
######################### 模块循环 ： 143 ##########################
AP：  0.8846933468800915
AUC SCORE:  0.8508737947658402
AP：  0.8847397034730051
AUC SCORE:  0.8510065140036731
######################### 模块循环 ： 144 ##########################
AP：  0.8848439970430122
AUC SCORE:  0.8511464072543617
AP：  0.8849973038807538
AUC SCORE:  0.8515015208907254
######################### 模块循环 ： 145 ##########################
AP：  0.8849306996772552
AUC SCORE:  0.8513759756657484
AP：  0.884905408021623
AUC SCORE:  0.851307822543618
######################### 模块循环 ： 146 ##########################
AP：  0.8850535352601727
AUC SCORE:  0.8512970615243343
AP：  0.8851573030637796
AUC SCORE:  0.851479998852158
######################### 模块循环 ： 147 ##########################
AP：  0.8851216634574863
AUC SCORE:  0.8513149965564739
AP：  0.8851626326984517
AUC SCORE:  0.8513472796143251
######################### 模块循环 ： 148 ##########################
AP：  0.8851982

AP：  0.889975724848197
AUC SCORE:  0.8562327823691461
AP：  0.8899478704671571
AUC SCORE:  0.8561718032598715
######################### 模块循环 ： 191 ##########################
AP：  0.8900428030505534
AUC SCORE:  0.8562865874655647
AP：  0.8900255576898776
AUC SCORE:  0.856239956382002
######################### 模块循环 ： 192 ##########################
AP：  0.8900010702381425
AUC SCORE:  0.8561825642791551
AP：  0.8899741161456485
AUC SCORE:  0.8561179981634528
######################### 模块循环 ： 193 ##########################
AP：  0.8900426153107813
AUC SCORE:  0.8562937614784205
AP：  0.8900971751430669
AUC SCORE:  0.8564336547291093
######################### 模块循环 ： 194 ##########################
AP：  0.8901390898071487
AUC SCORE:  0.8565197428833792
AP：  0.8900975140521745
AUC SCORE:  0.8564766988062442
######################### 模块循环 ： 195 ##########################
AP：  0.890094806807002
AUC SCORE:  0.8564300677226813
AP：  0.8900875551893621
AUC SCORE:  0.8563726756198347
#######################

...

AP：  0.8950613519567603
AUC SCORE:  0.866082702020202
AP：  0.8966468711145494
AUC SCORE:  0.8692213326446281
######################### 模块循环 ： 571 ##########################
AP：  0.8966939785668497
AUC SCORE:  0.8699853650137741
AP：  0.8967665859421439
AUC SCORE:  0.8695728592745637
######################### 模块循环 ： 572 ##########################
AP：  0.8968899738975532
AUC SCORE:  0.8705018939393938
AP：  0.8963421534512964
AUC SCORE:  0.8684070821854915
######################### 模块循环 ： 573 ##########################
AP：  0.8969123789784594
AUC SCORE:  0.8700786271808999
AP：  0.8963425722223517
AUC SCORE:  0.869311007805326
######################### 模块循环 ： 574 ##########################
AP：  0.8947351193297843
AUC SCORE:  0.8653330176767677
AP：  0.8951706257504253
AUC SCORE:  0.8667570592286502
######################### 模块循环 ： 575 ##########################
AP：  0.8947649958299141
AUC SCORE:  0.8664091196051424
AP：  0.8950309017302546
AUC SCORE:  0.8665884699265383
######################

AP：  0.8960849419877449
AUC SCORE:  0.8688267619375575
######################### 模块循环 ： 618 ##########################
AP：  0.8953542837895483
AUC SCORE:  0.8667857552800735
AP：  0.8961624250296173
AUC SCORE:  0.8689092630853994
######################### 模块循环 ： 619 ##########################
AP：  0.8955122575957828
AUC SCORE:  0.8671193468778696
AP：  0.8961244177470289
AUC SCORE:  0.8688195879247016
######################### 模块循环 ： 620 ##########################
AP：  0.8953293589455471
AUC SCORE:  0.8666889061065196
AP：  0.8962704297121187
AUC SCORE:  0.8690383953168044
######################### 模块循环 ： 621 ##########################
AP：  0.8959150416157569
AUC SCORE:  0.8681882747933884
AP：  0.8962218284662551
AUC SCORE:  0.8689630681818182
######################### 模块循环 ： 622 ##########################
AP：  0.8956190137567773
AUC SCORE:  0.867399133379247
AP：  0.8961971805903409
AUC SCORE:  0.8690061122589532
######################### 模块循环 ： 623 ##########################
AP：  0.89480

AP：  0.8954768729596625
AUC SCORE:  0.8675031565656566
AP：  0.8950703374353406
AUC SCORE:  0.8672090220385675
######################### 模块循环 ： 666 ##########################
AP：  0.8956918706145468
AUC SCORE:  0.868464474288338
AP：  0.8954812257210006
AUC SCORE:  0.8679622933884298
######################### 模块循环 ： 667 ##########################
AP：  0.8955658491596312
AUC SCORE:  0.86803044651056
AP：  0.8954614067474704
AUC SCORE:  0.8678833792470155
######################### 模块循环 ： 668 ##########################
AP：  0.8956175855463862
AUC SCORE:  0.868073490587695
AP：  0.8956306793592728
AUC SCORE:  0.8681021866391184
######################### 模块循环 ： 669 ##########################
AP：  0.8956823933949218
AUC SCORE:  0.8681344696969697
AP：  0.8957178867469623
AUC SCORE:  0.8682277318640954
######################### 模块循环 ： 670 ##########################
AP：  0.8957717620770109
AUC SCORE:  0.86828871097337
AP：  0.8958406691847263
AUC SCORE:  0.8684142561983472
######################### 

AP：  0.8954211981350638
AUC SCORE:  0.8671121728650139
######################### 模块循环 ： 713 ##########################
AP：  0.8953802254147992
AUC SCORE:  0.8682241448576675
AP：  0.8953579545237245
AUC SCORE:  0.8671193468778696
######################### 模块循环 ： 714 ##########################
AP：  0.8955445996466191
AUC SCORE:  0.868503931359045
AP：  0.8955547011856215
AUC SCORE:  0.8675497876492195
######################### 模块循环 ： 715 ##########################
AP：  0.8955474100335381
AUC SCORE:  0.8685613234618916
AP：  0.895541982586288
AUC SCORE:  0.8674672865013774
######################### 模块循环 ： 716 ##########################
AP：  0.8957683301631503
AUC SCORE:  0.8687765438475665
AP：  0.8956885376727561
AUC SCORE:  0.8681703397612488
######################### 模块循环 ： 717 ##########################
AP：  0.8958409438033976
AUC SCORE:  0.8689451331496787
AP：  0.8953363073366432
AUC SCORE:  0.8669220615243343
######################### 模块循环 ： 718 ##########################
AP：  0.896122

AP：  0.8949723169233124
AUC SCORE:  0.8673238062442608
AP：  0.8938750317608622
AUC SCORE:  0.8657921544995408
######################### 模块循环 ： 761 ##########################
AP：  0.8939515631378248
AUC SCORE:  0.8650891012396693
AP：  0.8948196847938324
AUC SCORE:  0.8677434859963269
######################### 模块循环 ： 762 ##########################
AP：  0.8946710737199886
AUC SCORE:  0.8668790174471993
AP：  0.8945983161349433
AUC SCORE:  0.8670798898071626
######################### 模块循环 ： 763 ##########################
AP：  0.8951474626499314
AUC SCORE:  0.868155991735537
AP：  0.894786325925629
AUC SCORE:  0.8674385904499541
######################### 模块循环 ： 764 ##########################
AP：  0.8949655523363389
AUC SCORE:  0.8676430498163452
AP：  0.894922339759197
AUC SCORE:  0.8676860938934803
######################### 模块循环 ： 765 ##########################
AP：  0.8949680590131559
AUC SCORE:  0.8675820707070707
AP：  0.8948788488005934
AUC SCORE:  0.8675569616620754
#######################

In [6]:
embedding_dim = 10
n_clusters = 7
alpha = 1

ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
ini_node_embeddings = ini_embedding.weight.detach()

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(ini_node_embeddings)
cluster_centers = kmeans.cluster_centers_
cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)

raw_nodes_embedding = ini_node_embeddings     # t=0时刻的节点嵌入
raw_cluster_centers = cluster_centers     # t=0时刻的类簇质心
for module_epoch in range(800):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
    raw_cluster_centers = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)

######################### 模块循环 ： 0 ##########################
AP：  0.5596937301839462
AUC SCORE:  0.5665963613406795
AP：  0.5840546440444696
AUC SCORE:  0.5949588211662076
######################### 模块循环 ： 1 ##########################
AP：  0.6038628253602232
AUC SCORE:  0.6111272526400366
AP：  0.6018755286468697
AUC SCORE:  0.6092960858585859
######################### 模块循环 ： 2 ##########################
AP：  0.609130503573724
AUC SCORE:  0.6113155704775023
AP：  0.6101357270290836
AUC SCORE:  0.610603549701561
######################### 模块循环 ： 3 ##########################
AP：  0.6143545520602425
AUC SCORE:  0.6143896349862259
AP：  0.6151923013092773
AUC SCORE:  0.6157275883838385
######################### 模块循环 ： 4 ##########################
AP：  0.6189991023722231
AUC SCORE:  0.6183281680440772
AP：  0.6212645741434647
AUC SCORE:  0.6197091655188246
######################### 模块循环 ： 5 ##########################
AP：  0.6250788125291016
AUC SCORE:  0.6225285525711662
AP：  0.6265342702919606
A

AP：  0.8019275925478765
AUC SCORE:  0.7392748507805327
######################### 模块循环 ： 48 ##########################
AP：  0.8026569983717116
AUC SCORE:  0.7392246326905416
AP：  0.8027563812636656
AUC SCORE:  0.7388049529384757
######################### 模块循环 ： 49 ##########################
AP：  0.8039413661012189
AUC SCORE:  0.7399420339761249
AP：  0.8043901715311289
AUC SCORE:  0.7405051939853078
######################### 模块循环 ： 50 ##########################
AP：  0.805436541866539
AUC SCORE:  0.7417391241965106
AP：  0.8060204558553944
AUC SCORE:  0.7424744605142333
######################### 模块循环 ： 51 ##########################
AP：  0.8074374122218813
AUC SCORE:  0.7442858987603306
AP：  0.8080068639639117
AUC SCORE:  0.7447557966023877
######################### 模块循环 ： 52 ##########################
AP：  0.8096656104794956
AUC SCORE:  0.7464022325528008
AP：  0.8101664993051952
AUC SCORE:  0.7467035410927456
######################### 模块循环 ： 53 ##########################
AP：  0.81097658183

AP：  0.8461152044274115
AUC SCORE:  0.7919608011937558
AP：  0.8463575514654952
AUC SCORE:  0.7922800447658402
######################### 模块循环 ： 96 ##########################
AP：  0.846632869961623
AUC SCORE:  0.792671028466483
AP：  0.846756726147933
AUC SCORE:  0.792757116620753
######################### 模块循环 ： 97 ##########################
AP：  0.8471588748900538
AUC SCORE:  0.792965162993572
AP：  0.8473424187057765
AUC SCORE:  0.7930620121671259
######################### 模块循环 ： 98 ##########################
AP：  0.8481460663172937
AUC SCORE:  0.7942349632690542
AP：  0.8484398005133933
AUC SCORE:  0.7944501836547291
######################### 模块循环 ： 99 ##########################
AP：  0.8488199981314323
AUC SCORE:  0.7954186753902663
AP：  0.8492262013047314
AUC SCORE:  0.7962221648301193
######################### 模块循环 ： 100 ##########################
AP：  0.8501164789392368
AUC SCORE:  0.7978399047291093
AP：  0.8505396691745442
AUC SCORE:  0.7986505681818181
######################### 模块循

AP：  0.8601731246288622
AUC SCORE:  0.8127295684113867
######################### 模块循环 ： 143 ##########################
AP：  0.8608664786677657
AUC SCORE:  0.8140352387511478
AP：  0.8610620664295765
AUC SCORE:  0.8143365472910927
######################### 模块循环 ： 144 ##########################
AP：  0.8611264570040167
AUC SCORE:  0.8143293732782368
AP：  0.8613276606850985
AUC SCORE:  0.8146808999081725
######################### 模块循环 ： 145 ##########################
AP：  0.8613870434321949
AUC SCORE:  0.8146557908631773
AP：  0.861462288076896
AUC SCORE:  0.8147777490817263
######################### 模块循环 ： 146 ##########################
AP：  0.8614241068797064
AUC SCORE:  0.8146486168503214
AP：  0.861460144956746
AUC SCORE:  0.8147992711202938
######################### 模块循环 ： 147 ##########################
AP：  0.8624160360109707
AUC SCORE:  0.8160332013314968
AP：  0.862644728328681
AUC SCORE:  0.8163129878328743
######################### 模块循环 ： 148 ##########################
AP：  0.8639786

AP：  0.8719592926384949
AUC SCORE:  0.8295884986225894
AP：  0.8719096505939232
AUC SCORE:  0.8295382805325987
######################### 模块循环 ： 191 ##########################
AP：  0.8724995572680453
AUC SCORE:  0.8308475378787878
AP：  0.87265579903985
AUC SCORE:  0.8311022153351699
######################### 模块循环 ： 192 ##########################
AP：  0.8732763703435675
AUC SCORE:  0.8319846189164372
AP：  0.8734674182589771
AUC SCORE:  0.83211733815427
######################### 模块循环 ： 193 ##########################
AP：  0.8733652795427206
AUC SCORE:  0.8318590736914601
AP：  0.8733618545006236
AUC SCORE:  0.8318949437557392
######################### 模块循环 ： 194 ##########################
AP：  0.873209375636918
AUC SCORE:  0.8315972222222222
AP：  0.8732485726639214
AUC SCORE:  0.8317335284664831
######################### 模块循环 ： 195 ##########################
AP：  0.8738445876017009
AUC SCORE:  0.83321854912764
AP：  0.8739660705849612
AUC SCORE:  0.833555727731864
######################### 模块

...

AP：  0.8909680258252758
AUC SCORE:  0.8610895890725435
AP：  0.8910327344959079
AUC SCORE:  0.8612241018135903
######################### 模块循环 ： 571 ##########################
AP：  0.8909983241880838
AUC SCORE:  0.8612474173553719
AP：  0.8909777885580372
AUC SCORE:  0.8612510043617998
######################### 模块循环 ： 572 ##########################
AP：  0.8910070105273107
AUC SCORE:  0.8612366563360883
AP：  0.8909771705268839
AUC SCORE:  0.8611577421946741
######################### 模块循环 ： 573 ##########################
AP：  0.8911422857583655
AUC SCORE:  0.861534377869605
AP：  0.8911354582145627
AUC SCORE:  0.8615666609274564
######################### 模块循环 ： 574 ##########################
AP：  0.8912622150112194
AUC SCORE:  0.8617639462809916
AP：  0.8912766282813809
AUC SCORE:  0.8617782943067034
######################### 模块循环 ： 575 ##########################
AP：  0.8911495025336538
AUC SCORE:  0.8615487258953168
AP：  0.891097776763271
AUC SCORE:  0.8614303546831956
######################

AP：  0.8925129614185018
AUC SCORE:  0.8635179924242424
######################### 模块循环 ： 618 ##########################
AP：  0.8923898716832996
AUC SCORE:  0.8633099460514233
AP：  0.8923289175976692
AUC SCORE:  0.8631700528007347
######################### 模块循环 ： 619 ##########################
AP：  0.8924971825992855
AUC SCORE:  0.8634354912764004
AP：  0.8924995047900537
AUC SCORE:  0.8634426652892563
######################### 模块循环 ： 620 ##########################
AP：  0.8924454616380746
AUC SCORE:  0.8633350550964187
AP：  0.8924862536754041
AUC SCORE:  0.8633063590449954
######################### 模块循环 ： 621 ##########################
AP：  0.892465832713044
AUC SCORE:  0.86319516184573
AP：  0.8924050643814819
AUC SCORE:  0.8631162477043159
######################### 模块循环 ： 622 ##########################
AP：  0.8924402276201179
AUC SCORE:  0.8631628787878787
AP：  0.8924493933541344
AUC SCORE:  0.8631270087235996
######################### 模块循环 ： 623 ##########################
AP：  0.8925572

AP：  0.8922750871723291
AUC SCORE:  0.8626893939393939
AP：  0.8921976108789439
AUC SCORE:  0.8625638487144169
######################### 模块循环 ： 666 ##########################
AP：  0.8921575354471555
AUC SCORE:  0.8624813475665749
AP：  0.8921689158794479
AUC SCORE:  0.8624131944444445
######################### 模块循环 ： 667 ##########################
AP：  0.8921742019072555
AUC SCORE:  0.8624921085858587
AP：  0.8921743454820232
AUC SCORE:  0.8624634125344353
######################### 模块循环 ： 668 ##########################
AP：  0.8921074068720101
AUC SCORE:  0.8624418904958677
AP：  0.8921482660058637
AUC SCORE:  0.8624741735537189
######################### 模块循环 ： 669 ##########################
AP：  0.8920662872930293
AUC SCORE:  0.862305584251607
AP：  0.892069044731982
AUC SCORE:  0.8622876492194673
######################### 模块循环 ： 670 ##########################
AP：  0.892088613277549
AUC SCORE:  0.8624131944444444
AP：  0.8920612841891871
AUC SCORE:  0.8623522153351699
#######################

AP：  0.8919123037054169
AUC SCORE:  0.8597193526170799
######################### 模块循环 ： 713 ##########################
AP：  0.8919079637573333
AUC SCORE:  0.8596691345270892
AP：  0.891905305496645
AUC SCORE:  0.8596045684113865
######################### 模块循环 ： 714 ##########################
AP：  0.8918840164557743
AUC SCORE:  0.859583046372819
AP：  0.8918807489324038
AUC SCORE:  0.8595328282828283
######################### 模块循环 ： 715 ##########################
AP：  0.8918462069314105
AUC SCORE:  0.8594754361799816
AP：  0.8918342153848763
AUC SCORE:  0.8593929350321396
######################### 模块循环 ： 716 ##########################
AP：  0.8917891142335486
AUC SCORE:  0.8592638028007347
AP：  0.8917885552113585
AUC SCORE:  0.8592099977043159
######################### 模块循环 ： 717 ##########################
AP：  0.8919040201622679
AUC SCORE:  0.8593893480257117
AP：  0.8919064645419301
AUC SCORE:  0.8593319559228649
######################### 模块循环 ： 718 ##########################
AP：  0.891880

AP：  0.895174085101524
AUC SCORE:  0.8653724747474748
AP：  0.8951985796545653
AUC SCORE:  0.8654478018824611
######################### 模块循环 ： 761 ##########################
AP：  0.8952279926416925
AUC SCORE:  0.865490845959596
AP：  0.8952386259787641
AUC SCORE:  0.8655051939853076
######################### 模块循环 ： 762 ##########################
AP：  0.8953238147919054
AUC SCORE:  0.865616391184573
AP：  0.8953358234684894
AUC SCORE:  0.8656773702938474
######################### 模块循环 ： 763 ##########################
AP：  0.8953080678392256
AUC SCORE:  0.8656199781910008
AP：  0.8953245691261076
AUC SCORE:  0.8656486742424243
######################### 模块循环 ： 764 ##########################
AP：  0.895280108799763
AUC SCORE:  0.8655625860881543
AP：  0.8952733272367399
AUC SCORE:  0.8655661730945822
######################### 模块循环 ： 765 ##########################
AP：  0.8953128381906525
AUC SCORE:  0.8656343262167128
AP：  0.8953013152417235
AUC SCORE:  0.8656379132231404
########################

In [7]:
embedding_dim = 8
n_clusters = 7
alpha = 1

ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
ini_node_embeddings = ini_embedding.weight.detach()

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(ini_node_embeddings)
cluster_centers = kmeans.cluster_centers_
cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)

raw_nodes_embedding = ini_node_embeddings     # t=0时刻的节点嵌入
raw_cluster_centers = cluster_centers     # t=0时刻的类簇质心
for module_epoch in range(800):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
    raw_cluster_centers = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)

######################### 模块循环 ： 0 ##########################
AP：  0.5344378997754846
AUC SCORE:  0.5251162190082644
AP：  0.5540706569353353
AUC SCORE:  0.5493069903581267
######################### 模块循环 ： 1 ##########################
AP：  0.5791626364842644
AUC SCORE:  0.5736161329201102
AP：  0.5754728097923782
AUC SCORE:  0.5708110938934803
######################### 模块循环 ： 2 ##########################
AP：  0.5875406401969718
AUC SCORE:  0.5851053145087236
AP：  0.5874458507241296
AUC SCORE:  0.5855716253443526
######################### 模块循环 ： 3 ##########################
AP：  0.5974739281726735
AUC SCORE:  0.5931832529843893
AP：  0.5993699097036721
AUC SCORE:  0.5948799070247934
######################### 模块循环 ： 4 ##########################
AP：  0.6013931724534217
AUC SCORE:  0.5978284263085399
AP：  0.6005860216601808
AUC SCORE:  0.5978140782828283
######################### 模块循环 ： 5 ##########################
AP：  0.604062910802621
AUC SCORE:  0.6005545511937557
AP：  0.6062057792322907


AP：  0.7714913779824094
AUC SCORE:  0.7111347853535352
######################### 模块循环 ： 48 ##########################
AP：  0.7731403429222556
AUC SCORE:  0.7125408918732782
AP：  0.7735829170809856
AUC SCORE:  0.7128386134067952
######################### 模块循环 ： 49 ##########################
AP：  0.775299364028513
AUC SCORE:  0.715396148989899
AP：  0.7760864009564941
AUC SCORE:  0.7165224690082644
######################### 模块循环 ： 50 ##########################
AP：  0.7778577964900943
AUC SCORE:  0.718409234389348
AP：  0.7788301404626901
AUC SCORE:  0.7194960973370065
######################### 模块循环 ： 51 ##########################
AP：  0.7805156112021328
AUC SCORE:  0.7207479625803489
AP：  0.7812469962274361
AUC SCORE:  0.7211030762167125
######################### 模块循环 ： 52 ##########################
AP：  0.7813965372833657
AUC SCORE:  0.7206798094582185
AP：  0.7814260458353413
AUC SCORE:  0.7205650252525252
######################### 模块循环 ： 53 ##########################
AP：  0.7834659521754

AP：  0.8312400033616503
AUC SCORE:  0.7815728305785123
AP：  0.8313274472505996
AUC SCORE:  0.7817127238292012
######################### 模块循环 ： 96 ##########################
AP：  0.8318383582288159
AUC SCORE:  0.782742194674013
AP：  0.8324444343503257
AUC SCORE:  0.7839043847566575
######################### 模块循环 ： 97 ##########################
AP：  0.8320103211323754
AUC SCORE:  0.7831260043617998
AP：  0.83201256044897
AUC SCORE:  0.7833591597796142
######################### 模块循环 ： 98 ##########################
AP：  0.8316889334661499
AUC SCORE:  0.7824731691919192
AP：  0.8318659975992839
AUC SCORE:  0.7827170856290175
######################### 模块循环 ： 99 ##########################
AP：  0.8325788424459921
AUC SCORE:  0.7834918790174472
AP：  0.8325358421766169
AUC SCORE:  0.783542097107438
######################### 模块循环 ： 100 ##########################
AP：  0.8330865509794662
AUC SCORE:  0.7843563475665748
AP：  0.8331942788357531
AUC SCORE:  0.7846325470615243
######################### 模块

AP：  0.8536079522019205
AUC SCORE:  0.8122166264921947
######################### 模块循环 ： 143 ##########################
AP：  0.8538647027129516
AUC SCORE:  0.8126793503213957
AP：  0.8539164451403412
AUC SCORE:  0.8127223943985309
######################### 模块循环 ： 144 ##########################
AP：  0.8538947071877946
AUC SCORE:  0.8124318468778696
AP：  0.8538402942236644
AUC SCORE:  0.812417498852158
######################### 模块循环 ： 145 ##########################
AP：  0.8539308911725438
AUC SCORE:  0.8123852157943068
AP：  0.8541128099240286
AUC SCORE:  0.8125322830578512
######################### 模块循环 ： 146 ##########################
AP：  0.854350920298201
AUC SCORE:  0.8125251090449953
AP：  0.8543426470376869
AUC SCORE:  0.8123385847107438
######################### 模块循环 ： 147 ##########################
AP：  0.8544871192607654
AUC SCORE:  0.8121771694214875
AP：  0.854604704418069
AUC SCORE:  0.8122560835629017
######################### 模块循环 ： 148 ##########################
AP：  0.8553317

AP：  0.86632896257495
AUC SCORE:  0.8268659607438016
AP：  0.8662156266633103
AUC SCORE:  0.826686610422406
######################### 模块循环 ： 191 ##########################
AP：  0.8660171154940819
AUC SCORE:  0.8262669306703398
AP：  0.865931101704287
AUC SCORE:  0.8261736685032139
######################### 模块循环 ： 192 ##########################
AP：  0.8657897253660993
AUC SCORE:  0.8258544249311295
AP：  0.8656679560870914
AUC SCORE:  0.8255746384297522
######################### 模块循环 ： 193 ##########################
AP：  0.8659450022185452
AUC SCORE:  0.8258436639118457
AP：  0.8660288828385501
AUC SCORE:  0.8259907311753902
######################### 模块循环 ： 194 ##########################
AP：  0.8659524954176477
AUC SCORE:  0.8259225780532599
AP：  0.8660501217362236
AUC SCORE:  0.8260983413682277
######################### 模块循环 ： 195 ##########################
AP：  0.8658044692484932
AUC SCORE:  0.8258257288797062
AP：  0.8658061830333061
AUC SCORE:  0.8258867079889807
########################

...

AP：  0.8850930456208074
AUC SCORE:  0.8544034090909091
AP：  0.8850476989865579
AUC SCORE:  0.8542563418273645
######################### 模块循环 ： 571 ##########################
AP：  0.8850820711977734
AUC SCORE:  0.8543567780073462
AP：  0.8850871779236564
AUC SCORE:  0.8543496039944903
######################### 模块循环 ： 572 ##########################
AP：  0.8851026010011366
AUC SCORE:  0.854363952020202
AP：  0.8851071525297918
AUC SCORE:  0.8543926480716253
######################### 模块循环 ： 573 ##########################
AP：  0.884969333819413
AUC SCORE:  0.8543460169880625
AP：  0.884913314192238
AUC SCORE:  0.8542599288337924
######################### 模块循环 ： 574 ##########################
AP：  0.8850716949812039
AUC SCORE:  0.8545540633608815
AP：  0.8851578781837777
AUC SCORE:  0.8546078684573003
######################### 模块循环 ： 575 ##########################
AP：  0.8852934170960023
AUC SCORE:  0.8549271120293847
AP：  0.8854550682303246
AUC SCORE:  0.8552929866850321
#######################

AP：  0.8868113593638406
AUC SCORE:  0.8582450929752066
######################### 模块循环 ： 618 ##########################
AP：  0.8868190509215231
AUC SCORE:  0.8580406336088154
AP：  0.8868014551591835
AUC SCORE:  0.8579581324609733
######################### 模块循环 ： 619 ##########################
AP：  0.886721685299779
AUC SCORE:  0.8575743227731865
AP：  0.88677055303145
AUC SCORE:  0.857542039715335
######################### 模块循环 ： 620 ##########################
AP：  0.8869136818327916
AUC SCORE:  0.8576066058310376
AP：  0.8870295142615534
AUC SCORE:  0.8577070420110193
######################### 模块循环 ： 621 ##########################
AP：  0.8869460116678973
AUC SCORE:  0.8573160583103765
AP：  0.8869131171165835
AUC SCORE:  0.8572909492653811
######################### 模块循环 ： 622 ##########################
AP：  0.8870065410042578
AUC SCORE:  0.8573734504132231
AP：  0.8870066058625804
AUC SCORE:  0.8572981232782368
######################### 模块循环 ： 623 ##########################
AP：  0.88699212

AP：  0.8892844206583416
AUC SCORE:  0.8605479511019284
AP：  0.8892504696427066
AUC SCORE:  0.8605120810376492
######################### 模块循环 ： 666 ##########################
AP：  0.8892472600972459
AUC SCORE:  0.8605084940312213
AP：  0.8892685703892941
AUC SCORE:  0.8605658861340678
######################### 模块循环 ： 667 ##########################
AP：  0.889339701954279
AUC SCORE:  0.860816976584022
AP：  0.8893472247881747
AUC SCORE:  0.8608743686868687
######################### 模块循环 ： 668 ##########################
AP：  0.8892536314301839
AUC SCORE:  0.8609066517447198
AP：  0.889254485168387
AUC SCORE:  0.8609174127640037
######################### 模块循环 ： 669 ##########################
AP：  0.8892496284667606
AUC SCORE:  0.8609999139118456
AP：  0.889187113790494
AUC SCORE:  0.8609676308539945
######################### 模块循环 ： 670 ##########################
AP：  0.8892714855141262
AUC SCORE:  0.8613048094582185
AP：  0.8892768817513075
AUC SCORE:  0.8612725264003673
########################

AP：  0.8890853166602803
AUC SCORE:  0.8609281737832875
######################### 模块循环 ： 713 ##########################
AP：  0.8892367955428714
AUC SCORE:  0.8609963269054178
AP：  0.8892741084482162
AUC SCORE:  0.8610214359504131
######################### 模块循环 ： 714 ##########################
AP：  0.8891883007488698
AUC SCORE:  0.8607380624426078
AP：  0.8890619682952116
AUC SCORE:  0.8605730601469237
######################### 模块循环 ： 715 ##########################
AP：  0.8891768910012978
AUC SCORE:  0.860773932506887
AP：  0.8891853419781209
AUC SCORE:  0.8607667584940313
######################### 模块循环 ： 716 ##########################
AP：  0.8891589247114428
AUC SCORE:  0.860519255050505
AP：  0.8892066214692991
AUC SCORE:  0.8604869719926539
######################### 模块循环 ： 717 ##########################
AP：  0.8891830507912327
AUC SCORE:  0.8605013200183654
AP：  0.8892300777626503
AUC SCORE:  0.8606734963269055
######################### 模块循环 ： 718 ##########################
AP：  0.889089

AP：  0.8898040040507702
AUC SCORE:  0.86099273989899
AP：  0.889712795633761
AUC SCORE:  0.8609174127640037
######################### 模块循环 ： 761 ##########################
AP：  0.8897214381492609
AUC SCORE:  0.8610429579889807
AP：  0.8897219687792768
AUC SCORE:  0.8610680670339762
######################### 模块循环 ： 762 ##########################
AP：  0.8898621420955619
AUC SCORE:  0.8612797004132231
AP：  0.8898727126500937
AUC SCORE:  0.8614339416896235
######################### 模块循环 ： 763 ##########################
AP：  0.8897305994999363
AUC SCORE:  0.8615523129017448
AP：  0.8896563923379567
AUC SCORE:  0.8615666609274564
######################### 模块循环 ： 764 ##########################
AP：  0.8896425920535722
AUC SCORE:  0.8614626377410469
AP：  0.889641046897125
AUC SCORE:  0.861419593663912
######################### 模块循环 ： 765 ##########################
AP：  0.8897572140900434
AUC SCORE:  0.8615953569788797
AP：  0.8898263821763801
AUC SCORE:  0.8616922061524335
#########################

In [8]:
embedding_dim = 10
n_clusters = 12
alpha = 1

ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
ini_node_embeddings = ini_embedding.weight.detach()

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(ini_node_embeddings)
cluster_centers = kmeans.cluster_centers_
cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)

raw_nodes_embedding = ini_node_embeddings     # t=0时刻的节点嵌入
raw_cluster_centers = cluster_centers     # t=0时刻的类簇质心
for module_epoch in range(800):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
    raw_cluster_centers = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)

######################### 模块循环 ： 0 ##########################
AP：  0.5389925452874272
AUC SCORE:  0.5454868285123968
AP：  0.5462358820339592
AUC SCORE:  0.553582702020202
######################### 模块循环 ： 1 ##########################
AP：  0.5522100881982881
AUC SCORE:  0.560290404040404
AP：  0.5623667717404173
AUC SCORE:  0.5687324236685032
######################### 模块循环 ： 2 ##########################
AP：  0.5783493258261534
AUC SCORE:  0.5814142848943985
AP：  0.5780662650208983
AUC SCORE:  0.5797570879247016
######################### 模块循环 ： 3 ##########################
AP：  0.5909549611650542
AUC SCORE:  0.5847681359044996
AP：  0.5942319052114169
AUC SCORE:  0.5867374024334252
######################### 模块循环 ： 4 ##########################
AP：  0.5920472277803486
AUC SCORE:  0.5870745810376492
AP：  0.5936103712698351
AUC SCORE:  0.5868665346648301
######################### 模块循环 ： 5 ##########################
AP：  0.5967892564466191
AUC SCORE:  0.5867912075298438
AP：  0.5909256699475302
A

AP：  0.8142686832315126
AUC SCORE:  0.7647605314508724
######################### 模块循环 ： 48 ##########################
AP：  0.8154967241105905
AUC SCORE:  0.7659155475206612
AP：  0.8161378143357325
AUC SCORE:  0.766636535812672
######################### 模块循环 ： 49 ##########################
AP：  0.8173269684145568
AUC SCORE:  0.7679135101010102
AP：  0.8179896513711004
AUC SCORE:  0.7684874311294766
######################### 模块循环 ： 50 ##########################
AP：  0.8192023329467246
AUC SCORE:  0.7695994031221305
AP：  0.8199428272067368
AUC SCORE:  0.770338326446281
######################### 模块循环 ： 51 ##########################
AP：  0.8211735176451096
AUC SCORE:  0.7715973657024793
AP：  0.8216451083450103
AUC SCORE:  0.7720313934802571
######################### 模块循环 ： 52 ##########################
AP：  0.8230013536779883
AUC SCORE:  0.7733944559228649
AP：  0.8235695388577463
AUC SCORE:  0.7739002238292011
######################### 模块循环 ： 53 ##########################
AP：  0.824817252907

AP：  0.8641786996170253
AUC SCORE:  0.8231785181359045
AP：  0.8643439245066629
AUC SCORE:  0.8235623278236914
######################### 模块循环 ： 96 ##########################
AP：  0.864724497140811
AUC SCORE:  0.8241075528007347
AP：  0.8648423096356237
AUC SCORE:  0.8243729912764004
######################### 模块循环 ： 97 ##########################
AP：  0.8651460019393686
AUC SCORE:  0.8246707128099173
AP：  0.8651943708561799
AUC SCORE:  0.8248464761248853
######################### 模块循环 ： 98 ##########################
AP：  0.865458318223194
AUC SCORE:  0.8250975665748392
AP：  0.8654825577966209
AUC SCORE:  0.8252087637741048
######################### 模块循环 ： 99 ##########################
AP：  0.8660524521066175
AUC SCORE:  0.8260624713039486
AP：  0.8662354727658608
AUC SCORE:  0.8263458448117539
######################### 模块循环 ： 100 ##########################
AP：  0.866699445017864
AUC SCORE:  0.8270919421487604
AP：  0.8668394130922376
AUC SCORE:  0.8272928145087236
######################### 模

AP：  0.8807377515459355
AUC SCORE:  0.8452601297061525
######################### 模块循环 ： 143 ##########################
AP：  0.8811889002912956
AUC SCORE:  0.8451058884297521
AP：  0.8817347835258907
AUC SCORE:  0.8463756887052343
######################### 模块循环 ： 144 ##########################
AP：  0.8809493379676284
AUC SCORE:  0.8450449093204775
AP：  0.8818687648349861
AUC SCORE:  0.8469316747015612
######################### 模块循环 ： 145 ##########################
AP：  0.8812547934198698
AUC SCORE:  0.845661874426079
AP：  0.8814549778661727
AUC SCORE:  0.8462214474288338
######################### 模块循环 ： 146 ##########################
AP：  0.8812173063432762
AUC SCORE:  0.8457587235996327
AP：  0.8818024337369176
AUC SCORE:  0.8475988578971534
######################### 模块循环 ： 147 ##########################
AP：  0.8812061122992395
AUC SCORE:  0.8460313360881543
AP：  0.8821193776125382
AUC SCORE:  0.8480185376492195
######################### 模块循环 ： 148 ##########################
AP：  0.88120

AP：  0.8849000991645607
AUC SCORE:  0.8546078684573003
AP：  0.8812183301899575
AUC SCORE:  0.8484023473370064
######################### 模块循环 ： 191 ##########################
AP：  0.8845560002152459
AUC SCORE:  0.8528896923783287
AP：  0.8856633747703136
AUC SCORE:  0.8551459194214877
######################### 模块循环 ： 192 ##########################
AP：  0.8840198871195313
AUC SCORE:  0.8524413165748393
AP：  0.8852356751440197
AUC SCORE:  0.8543890610651974
######################### 模块循环 ： 193 ##########################
AP：  0.8837627786358598
AUC SCORE:  0.8527605601469239
AP：  0.8836401637716043
AUC SCORE:  0.8518996786042241
######################### 模块循环 ： 194 ##########################
AP：  0.8844900556140838
AUC SCORE:  0.8527390381083562
AP：  0.8851514558250445
AUC SCORE:  0.8541738406795225
######################### 模块循环 ： 195 ##########################
AP：  0.8846500464488269
AUC SCORE:  0.853130021808999
AP：  0.885149064831174
AUC SCORE:  0.8542922118916438
######################

...

AP：  0.898272805325603
AUC SCORE:  0.8762590392561984
AP：  0.8964356186379623
AUC SCORE:  0.8728800792011018
######################### 模块循环 ： 571 ##########################
AP：  0.8967121039084367
AUC SCORE:  0.8743615128558311
AP：  0.8963764627242095
AUC SCORE:  0.8733176939853076
######################### 模块循环 ： 572 ##########################
AP：  0.8977916091900676
AUC SCORE:  0.8756026170798898
AP：  0.8966194554923532
AUC SCORE:  0.8730379074839303
######################### 模块循环 ： 573 ##########################
AP：  0.8975146421691518
AUC SCORE:  0.8755739210284665
AP：  0.8963147491389118
AUC SCORE:  0.873636937557392
######################### 模块循环 ： 574 ##########################
AP：  0.8991309481172554
AUC SCORE:  0.877970041322314
AP：  0.8968762032423045
AUC SCORE:  0.8741319444444444
######################### 模块循环 ： 575 ##########################
AP：  0.8986393598671527
AUC SCORE:  0.877550361570248
AP：  0.8960887394615473
AUC SCORE:  0.8727007288797061
########################

AP：  0.8976824489257971
AUC SCORE:  0.876786329201102
######################### 模块循环 ： 618 ##########################
AP：  0.8985401406187469
AUC SCORE:  0.8774355773645546
AP：  0.8972901296807079
AUC SCORE:  0.8749246728650137
######################### 模块循环 ： 619 ##########################
AP：  0.895592593359448
AUC SCORE:  0.872919536271809
AP：  0.8963329302747004
AUC SCORE:  0.873938246097337
######################### 模块循环 ： 620 ##########################
AP：  0.8957039466328214
AUC SCORE:  0.8723671372819101
AP：  0.8961957029074078
AUC SCORE:  0.8731239956382002
######################### 模块循环 ： 621 ##########################
AP：  0.8986756681753656
AUC SCORE:  0.877249053030303
AP：  0.8988885217107211
AUC SCORE:  0.8766607839761249
######################### 模块循环 ： 622 ##########################
AP：  0.897035025358083
AUC SCORE:  0.8743686868686869
AP：  0.899508312379932
AUC SCORE:  0.8779879763544536
######################### 模块循环 ： 623 ##########################
AP：  0.89924709354

AP：  0.900311572113561
AUC SCORE:  0.879924959825528
AP：  0.8997354586705832
AUC SCORE:  0.8781386306244261
######################### 模块循环 ： 666 ##########################
AP：  0.9004944406122669
AUC SCORE:  0.8800469180440771
AP：  0.9006607893814703
AUC SCORE:  0.8802334423783288
######################### 模块循环 ： 667 ##########################
AP：  0.9003518484341837
AUC SCORE:  0.8799715909090908
AP：  0.9001572497410039
AUC SCORE:  0.8791071223599631
######################### 模块循环 ： 668 ##########################
AP：  0.900543041271493
AUC SCORE:  0.8802370293847566
AP：  0.8997808388095639
AUC SCORE:  0.8784614612029386
######################### 模块循环 ： 669 ##########################
AP：  0.9005238343287396
AUC SCORE:  0.8801330061983471
AP：  0.9004879628764773
AUC SCORE:  0.8798532196969697
######################### 模块循环 ： 670 ##########################
AP：  0.9003654355445625
AUC SCORE:  0.8800146349862259
AP：  0.8992265946543252
AUC SCORE:  0.8771916609274564
#######################

AP：  0.899564216839712
AUC SCORE:  0.8795232151056015
######################### 模块循环 ： 713 ##########################
AP：  0.8993225843193315
AUC SCORE:  0.8797743055555556
AP：  0.8998898084122939
AUC SCORE:  0.8800469180440771
######################### 模块循环 ： 714 ##########################
AP：  0.8990809037646241
AUC SCORE:  0.8794837580348942
AP：  0.8993879190911132
AUC SCORE:  0.8790066861799816
######################### 模块循环 ： 715 ##########################
AP：  0.8989162236277901
AUC SCORE:  0.879038969237833
AP：  0.9002626461566476
AUC SCORE:  0.8810584538567494
######################### 模块循环 ： 716 ##########################
AP：  0.8987224718540988
AUC SCORE:  0.8792183195592287
AP：  0.8994550706200606
AUC SCORE:  0.8794658230027548
######################### 模块循环 ： 717 ##########################
AP：  0.8981390313700861
AUC SCORE:  0.8777297118916437
AP：  0.9001063059561936
AUC SCORE:  0.8805598599632691
######################### 模块循环 ： 718 ##########################
AP：  0.899173

AP：  0.901874319469391
AUC SCORE:  0.8851655762167125
AP：  0.899906432235876
AUC SCORE:  0.8811050849403121
######################### 模块循环 ： 761 ##########################
AP：  0.9001402323074166
AUC SCORE:  0.8823174931129476
AP：  0.8992240856202619
AUC SCORE:  0.8805132288797062
######################### 模块循环 ： 762 ##########################
AP：  0.9001855999422713
AUC SCORE:  0.8823820592286501
AP：  0.8997200074632523
AUC SCORE:  0.8812557392102847
######################### 模块循环 ： 763 ##########################
AP：  0.9002313969605535
AUC SCORE:  0.882765868916437
AP：  0.8993218867945998
AUC SCORE:  0.8804773588154271
######################### 模块循环 ： 764 ##########################
AP：  0.8996088343680604
AUC SCORE:  0.8816503099173555
AP：  0.8994904767905627
AUC SCORE:  0.8807571453168044
######################### 模块循环 ： 765 ##########################
AP：  0.8994088834588314
AUC SCORE:  0.8815247646923785
AP：  0.8989665533834625
AUC SCORE:  0.8797886535812672
#######################

In [9]:
embedding_dim = 8
n_clusters = 12
alpha = 1

ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
ini_node_embeddings = ini_embedding.weight.detach()

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(ini_node_embeddings)
cluster_centers = kmeans.cluster_centers_
cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)

raw_nodes_embedding = ini_node_embeddings     # t=0时刻的节点嵌入
raw_cluster_centers = cluster_centers     # t=0时刻的类簇质心
for module_epoch in range(800):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
    raw_cluster_centers = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)

######################### 模块循环 ： 0 ##########################
AP：  0.530230827035407
AUC SCORE:  0.5369784492653811
AP：  0.5382106496461001
AUC SCORE:  0.54467257805326
######################### 模块循环 ： 1 ##########################
AP：  0.574583240927604
AUC SCORE:  0.5802413337924701
AP：  0.5785689325602352
AUC SCORE:  0.5856110824150597
######################### 模块循环 ： 2 ##########################
AP：  0.5812792951106727
AUC SCORE:  0.5850264003673094
AP：  0.5850245037696237
AUC SCORE:  0.590690283516988
######################### 模块循环 ： 3 ##########################
AP：  0.5921935547932258
AUC SCORE:  0.5968563475665749
AP：  0.5891636394069971
AUC SCORE:  0.5914955664600551
######################### 模块循环 ： 4 ##########################
AP：  0.5956611097838017
AUC SCORE:  0.5988507231404958
AP：  0.5934715535531496
AUC SCORE:  0.5963900367309458
######################### 模块循环 ： 5 ##########################
AP：  0.5974246432263798
AUC SCORE:  0.5996255165289257
AP：  0.599335602887882
AUC S

AP：  0.8094782619746455
AUC SCORE:  0.7597602444903582
######################### 模块循环 ： 48 ##########################
AP：  0.8106933750843243
AUC SCORE:  0.7609726526629934
AP：  0.811479435458319
AUC SCORE:  0.7616864669421487
######################### 模块循环 ： 49 ##########################
AP：  0.8127047353641964
AUC SCORE:  0.7628630050505051
AP：  0.8134306776828727
AUC SCORE:  0.7635732323232325
######################### 模块循环 ： 50 ##########################
AP：  0.8145210853457265
AUC SCORE:  0.7646959653351699
AP：  0.8152470030286605
AUC SCORE:  0.7654779327364554
######################### 模块循环 ： 51 ##########################
AP：  0.8164674898517494
AUC SCORE:  0.7667046889348026
AP：  0.8172695663001319
AUC SCORE:  0.7675045913682277
######################### 模块循环 ： 52 ##########################
AP：  0.8183462708786137
AUC SCORE:  0.7685914543158863
AP：  0.8189241590501365
AUC SCORE:  0.7689142848943985
######################### 模块循环 ： 53 ##########################
AP：  0.81997808558

AP：  0.8588915435386297
AUC SCORE:  0.8132496843434344
AP：  0.8590399820847414
AUC SCORE:  0.8134075126262627
######################### 模块循环 ： 96 ##########################
AP：  0.8591332569331778
AUC SCORE:  0.8132676193755739
AP：  0.8591772457065163
AUC SCORE:  0.8132030532598714
######################### 模块循环 ： 97 ##########################
AP：  0.8593375057706815
AUC SCORE:  0.8134003386134068
AP：  0.8594615531360457
AUC SCORE:  0.8136191460055096
######################### 模块循环 ： 98 ##########################
AP：  0.8594621800634618
AUC SCORE:  0.8135868629476583
AP：  0.8595296494412648
AUC SCORE:  0.8136765381083562
######################### 模块循环 ： 99 ##########################
AP：  0.8596274978662548
AUC SCORE:  0.8137195821854912
AP：  0.8597665287090435
AUC SCORE:  0.8138953455004589
######################### 模块循环 ： 100 ##########################
AP：  0.8598470179216027
AUC SCORE:  0.8140495867768596
AP：  0.859948299031168
AUC SCORE:  0.8141607839761249
#########################

AP：  0.8702526708783732
AUC SCORE:  0.8309121039944904
######################### 模块循环 ： 143 ##########################
AP：  0.8662588264293971
AUC SCORE:  0.8256571395775942
AP：  0.8679462082117855
AUC SCORE:  0.8287455521120294
######################### 模块循环 ： 144 ##########################
AP：  0.8694957805794938
AUC SCORE:  0.8291831668962351
AP：  0.8690300095639805
AUC SCORE:  0.8293625172176309
######################### 模块循环 ： 145 ##########################
AP：  0.8686808959446548
AUC SCORE:  0.8294701274104683
AP：  0.867844803124256
AUC SCORE:  0.8308331898530761
######################### 模块循环 ： 146 ##########################
AP：  0.8701233601257357
AUC SCORE:  0.8328741965105602
AP：  0.869086069959673
AUC SCORE:  0.8317514634986225
######################### 模块循环 ： 147 ##########################
AP：  0.8691925119669011
AUC SCORE:  0.8321747302571165
AP：  0.8645464507911912
AUC SCORE:  0.8227014462809916
######################### 模块循环 ： 148 ##########################
AP：  0.863727

AP：  0.8687303090289
AUC SCORE:  0.830668187557392
AP：  0.8687319098272897
AUC SCORE:  0.8305569903581267
######################### 模块循环 ： 191 ##########################
AP：  0.8684525877989248
AUC SCORE:  0.8301444846189164
AP：  0.8685886527665769
AUC SCORE:  0.830036874426079
######################### 模块循环 ： 192 ##########################
AP：  0.8682101949674172
AUC SCORE:  0.8295024104683196
AP：  0.8682746369756476
AUC SCORE:  0.8295418675390265
######################### 模块循环 ： 193 ##########################
AP：  0.8680919738423264
AUC SCORE:  0.8292836030762167
AP：  0.8687384110632675
AUC SCORE:  0.8306107954545454
######################### 模块循环 ： 194 ##########################
AP：  0.8633632036583723
AUC SCORE:  0.8251441976584022
AP：  0.8643113148787306
AUC SCORE:  0.8251621326905417
######################### 模块循环 ： 195 ##########################
AP：  0.8594129522105572
AUC SCORE:  0.8167039715335169
AP：  0.866649086410928
AUC SCORE:  0.8252553948576675
######################### 

...

AP：  0.8794198143343885
AUC SCORE:  0.8593032598714416
AP：  0.8783494691357617
AUC SCORE:  0.8568784435261707
######################### 模块循环 ： 571 ##########################
AP：  0.8795976949701957
AUC SCORE:  0.8594861971992654
AP：  0.8786597452212794
AUC SCORE:  0.8574057334710744
######################### 模块循环 ： 572 ##########################
AP：  0.8796936181928516
AUC SCORE:  0.8594682621671258
AP：  0.8788323185492503
AUC SCORE:  0.8574559515610651
######################### 模块循环 ： 573 ##########################
AP：  0.8796662807426059
AUC SCORE:  0.8595292412764004
AP：  0.8788506188340605
AUC SCORE:  0.8577249770431589
######################### 模块循环 ： 574 ##########################
AP：  0.8797611903137763
AUC SCORE:  0.8595435893021119
AP：  0.8790052536193753
AUC SCORE:  0.8576819329660239
######################### 模块循环 ： 575 ##########################
AP：  0.8797296825269776
AUC SCORE:  0.8596153294306703
AP：  0.8789968913805577
AUC SCORE:  0.8580765036730946
####################

AP：  0.8722727368896426
AUC SCORE:  0.8507195534894398
######################### 模块循环 ： 618 ##########################
AP：  0.8788935475402876
AUC SCORE:  0.859586633379247
AP：  0.8767163215605634
AUC SCORE:  0.8558417986685032
######################### 模块循环 ： 619 ##########################
AP：  0.8745102158238097
AUC SCORE:  0.853980142332415
AP：  0.8750275635644681
AUC SCORE:  0.8541917757116622
######################### 模块循环 ： 620 ##########################
AP：  0.8774243191806933
AUC SCORE:  0.8585930325987144
AP：  0.8786091728271646
AUC SCORE:  0.8581625918273647
######################### 模块循环 ： 621 ##########################
AP：  0.8780157040166054
AUC SCORE:  0.8578397612488522
AP：  0.877750429635589
AUC SCORE:  0.8572766012396694
######################### 模块循环 ： 622 ##########################
AP：  0.8788470299522063
AUC SCORE:  0.8593857610192837
AP：  0.8784294206100814
AUC SCORE:  0.8577680211202938
######################### 模块循环 ： 623 ##########################
AP：  0.8789191

AP：  0.8788789100986263
AUC SCORE:  0.8593032598714417
AP：  0.8786796050580861
AUC SCORE:  0.8579366104224059
######################### 模块循环 ： 666 ##########################
AP：  0.8785210018246772
AUC SCORE:  0.8586647727272728
AP：  0.8751806904355887
AUC SCORE:  0.8536860078053259
######################### 模块循环 ： 667 ##########################
AP：  0.8765804854139841
AUC SCORE:  0.8553826618457301
AP：  0.8785303509837467
AUC SCORE:  0.8579868285123966
######################### 模块循环 ： 668 ##########################
AP：  0.877564582221281
AUC SCORE:  0.8580800906795225
AP：  0.8772454288037637
AUC SCORE:  0.8569412161386594
######################### 模块循环 ： 669 ##########################
AP：  0.8790016516200677
AUC SCORE:  0.8598628328741965
AP：  0.8768507494248148
AUC SCORE:  0.8560139749770431
######################### 模块循环 ： 670 ##########################
AP：  0.8791878017268188
AUC SCORE:  0.8597121786042239
AP：  0.8760508084916864
AUC SCORE:  0.8545002582644627
#####################

AP：  0.8786906809264229
AUC SCORE:  0.8579330234159779
######################### 模块循环 ： 713 ##########################
AP：  0.879548751324795
AUC SCORE:  0.8598987029384757
AP：  0.8788106365861148
AUC SCORE:  0.8581625918273645
######################### 模块循环 ： 714 ##########################
AP：  0.8795139113760406
AUC SCORE:  0.8599417470156108
AP：  0.87864154652558
AUC SCORE:  0.8578541092745638
######################### 模块循环 ： 715 ##########################
AP：  0.8794975904874281
AUC SCORE:  0.8598664198806245
AP：  0.8787070326872393
AUC SCORE:  0.8580406336088153
######################### 模块循环 ： 716 ##########################
AP：  0.879524822703748
AUC SCORE:  0.860017074150597
AP：  0.878632927156096
AUC SCORE:  0.8578792183195593
######################### 模块循环 ： 717 ##########################
AP：  0.8794322833296555
AUC SCORE:  0.8598269628099173
AP：  0.8786100297560551
AUC SCORE:  0.8578110651974288
######################### 模块循环 ： 718 ##########################
AP：  0.8795036061

AP：  0.8799182028595924
AUC SCORE:  0.862136994949495
AP：  0.8798394996870584
AUC SCORE:  0.8617029671717172
######################### 模块循环 ： 761 ##########################
AP：  0.8801790050220918
AUC SCORE:  0.862129820936639
AP：  0.8804152001310661
AUC SCORE:  0.8614518767217632
######################### 模块循环 ： 762 ##########################
AP：  0.881332857117615
AUC SCORE:  0.8630947256657485
AP：  0.881058078485759
AUC SCORE:  0.8624921085858586
######################### 模块循环 ： 763 ##########################
AP：  0.8813718416193272
AUC SCORE:  0.8633745121671258
AP：  0.881225741421419
AUC SCORE:  0.862768308080808
######################### 模块循环 ： 764 ##########################
AP：  0.8783755389546237
AUC SCORE:  0.8604654499540862
AP：  0.8784939932526643
AUC SCORE:  0.860350665748393
######################### 模块循环 ： 765 ##########################
AP：  0.8797632271152432
AUC SCORE:  0.8622087350780532
AP：  0.8786342534362166
AUC SCORE:  0.8602287075298438
######################### 模

In [10]:
embedding_dim = 10
n_clusters = 24
alpha = 1

ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
ini_node_embeddings = ini_embedding.weight.detach()

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(ini_node_embeddings)
cluster_centers = kmeans.cluster_centers_
cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)

raw_nodes_embedding = ini_node_embeddings     # t=0时刻的节点嵌入
raw_cluster_centers = cluster_centers     # t=0时刻的类簇质心
for module_epoch in range(800):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
    raw_cluster_centers = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)

######################### 模块循环 ： 0 ##########################
AP：  0.5646340341659302
AUC SCORE:  0.5738851584022039
AP：  0.5633580639646891
AUC SCORE:  0.5684831267217632
######################### 模块循环 ： 1 ##########################
AP：  0.5719970524376369
AUC SCORE:  0.5840363865932048
AP：  0.579401086274357
AUC SCORE:  0.5924263946280992
######################### 模块循环 ： 2 ##########################
AP：  0.5901480801032968
AUC SCORE:  0.5968276515151515
AP：  0.5803143947199703
AUC SCORE:  0.5852165117079889
######################### 模块循环 ： 3 ##########################
AP：  0.5806474414046979
AUC SCORE:  0.5837960571625345
AP：  0.5815924114736296
AUC SCORE:  0.5891729797979798
######################### 模块循环 ： 4 ##########################
AP：  0.5810993585868021
AUC SCORE:  0.592175304178145
AP：  0.5794459741784226
AUC SCORE:  0.5840327995867769
######################### 模块循环 ： 5 ##########################
AP：  0.5865209070968087
AUC SCORE:  0.5940943526170798
AP：  0.5805800703935255
A

AP：  0.7767600756251302
AUC SCORE:  0.7417104281450873
######################### 模块循环 ： 48 ##########################
AP：  0.7781164764332347
AUC SCORE:  0.7426000057392104
AP：  0.7788541297313238
AUC SCORE:  0.7432743629476585
######################### 模块循环 ： 49 ##########################
AP：  0.7802406074843433
AUC SCORE:  0.7442643767217632
AP：  0.7807227945217955
AUC SCORE:  0.7446625344352618
######################### 模块循环 ： 50 ##########################
AP：  0.7819347724899689
AUC SCORE:  0.7455521120293847
AP：  0.7827075722469027
AUC SCORE:  0.7463161443985308
######################### 模块循环 ： 51 ##########################
AP：  0.7839525419360467
AUC SCORE:  0.7472021349862258
AP：  0.7845174677935632
AUC SCORE:  0.7476720328282828
######################### 模块循环 ： 52 ##########################
AP：  0.785907765735141
AUC SCORE:  0.7487015036730946
AP：  0.7865000585106172
AUC SCORE:  0.7491821625344353
######################### 模块循环 ： 53 ##########################
AP：  0.78767450684

AP：  0.8335206578887779
AUC SCORE:  0.7932341884756657
AP：  0.8336552147075889
AUC SCORE:  0.7934027777777777
######################### 模块循环 ： 96 ##########################
AP：  0.8340426691064404
AUC SCORE:  0.7938475665748392
AP：  0.8341469992743317
AUC SCORE:  0.7940305039026628
######################### 模块循环 ： 97 ##########################
AP：  0.8343408624268644
AUC SCORE:  0.7943712695133149
AP：  0.8345057017097847
AUC SCORE:  0.7945721418732783
######################### 模块循环 ： 98 ##########################
AP：  0.8347949333430251
AUC SCORE:  0.7948949724517906
AP：  0.8349664513194865
AUC SCORE:  0.7951101928374655
######################### 模块循环 ： 99 ##########################
AP：  0.8351755236019074
AUC SCORE:  0.7953433482552801
AP：  0.83517731499933
AUC SCORE:  0.7953648702938475
######################### 模块循环 ： 100 ##########################
AP：  0.8353255159424473
AUC SCORE:  0.7956482438016528
AP：  0.8354898547087963
AUC SCORE:  0.7957737890266301
######################### 

AP：  0.8516008662799117
AUC SCORE:  0.8149032943067035
######################### 模块循环 ： 143 ##########################
AP：  0.8521774394215449
AUC SCORE:  0.8159937442607896
AP：  0.8518127340024831
AUC SCORE:  0.8152619949494949
######################### 模块循环 ： 144 ##########################
AP：  0.85243968718426
AUC SCORE:  0.8163632059228649
AP：  0.851969995881299
AUC SCORE:  0.8154951503673094
######################### 模块循环 ： 145 ##########################
AP：  0.852523687029948
AUC SCORE:  0.8165533172635446
AP：  0.8520512041154834
AUC SCORE:  0.8156135215794306
######################### 模块循环 ： 146 ##########################
AP：  0.8526510923800572
AUC SCORE:  0.816700384527089
AP：  0.8525440634483987
AUC SCORE:  0.8163452708907253
######################### 模块循环 ： 147 ##########################
AP：  0.8531521378983007
AUC SCORE:  0.8175792011019284
AP：  0.852705995745475
AUC SCORE:  0.8168223427456383
######################### 模块循环 ： 148 ##########################
AP：  0.8531295416

AP：  0.8564211536018997
AUC SCORE:  0.8246384297520661
AP：  0.8565230354512856
AUC SCORE:  0.8242044019742882
######################### 模块循环 ： 191 ##########################
AP：  0.8563414543700428
AUC SCORE:  0.8246563647842058
AP：  0.8564724611090513
AUC SCORE:  0.8242474460514233
######################### 模块循环 ： 192 ##########################
AP：  0.856412236748354
AUC SCORE:  0.8248070190541782
AP：  0.8566229328463625
AUC SCORE:  0.8244662534435261
######################### 模块循环 ： 193 ##########################
AP：  0.856631941961184
AUC SCORE:  0.825065283516988
AP：  0.8569234853757954
AUC SCORE:  0.8249110422405878
######################### 模块循环 ： 194 ##########################
AP：  0.8568276699340983
AUC SCORE:  0.825323547979798
AP：  0.856916421390291
AUC SCORE:  0.8247388659320477
######################### 模块循环 ： 195 ##########################
AP：  0.8570905502130687
AUC SCORE:  0.8255710514233241
AP：  0.8574870897008156
AUC SCORE:  0.8257611627640036
#########################

...

AP：  0.8650174523030555
AUC SCORE:  0.8557592975206612
AP：  0.8659319561621631
AUC SCORE:  0.8571008379247016
######################### 模块循环 ： 571 ##########################
AP：  0.8654657813032547
AUC SCORE:  0.8564085456841137
AP：  0.8659311917131749
AUC SCORE:  0.8576066058310376
######################### 模块循环 ： 572 ##########################
AP：  0.8656032623962335
AUC SCORE:  0.8566775711662075
AP：  0.8669032538968662
AUC SCORE:  0.8585930325987144
######################### 模块循环 ： 573 ##########################
AP：  0.865269707968857
AUC SCORE:  0.8564623507805327
AP：  0.865557021073833
AUC SCORE:  0.8564946338383839
######################### 模块循环 ： 574 ##########################
AP：  0.863684151205738
AUC SCORE:  0.8554615759871441
AP：  0.8635981530535812
AUC SCORE:  0.8556122302571166
######################### 模块循环 ： 575 ##########################
AP：  0.8624988998653529
AUC SCORE:  0.8535783976124885
AP：  0.8640418668794764
AUC SCORE:  0.8563116965105602
#######################

AP：  0.8643050685812966
AUC SCORE:  0.8569071395775942
######################### 模块循环 ： 618 ##########################
AP：  0.8642449487680042
AUC SCORE:  0.8563941976584022
AP：  0.8645489315016679
AUC SCORE:  0.8566452881083563
######################### 模块循环 ： 619 ##########################
AP：  0.8641297344867352
AUC SCORE:  0.8563224575298439
AP：  0.8642021496055037
AUC SCORE:  0.8566309400826446
######################### 模块循环 ： 620 ##########################
AP：  0.8641453922491129
AUC SCORE:  0.856275826446281
AP：  0.8643756362357851
AUC SCORE:  0.8564731117998163
######################### 模块循环 ： 621 ##########################
AP：  0.8641637455379754
AUC SCORE:  0.8562399563820018
AP：  0.864218812654639
AUC SCORE:  0.8564228937098255
######################### 模块循环 ： 622 ##########################
AP：  0.8641279528674564
AUC SCORE:  0.8562543044077136
AP：  0.8644335628491043
AUC SCORE:  0.8565089818640956
######################### 模块循环 ： 623 ##########################
AP：  0.864128

AP：  0.8597485079566682
AUC SCORE:  0.853047520661157
AP：  0.8606869056489195
AUC SCORE:  0.8544213441230486
######################### 模块循环 ： 666 ##########################
AP：  0.8580040447263098
AUC SCORE:  0.8491340966483012
AP：  0.8622550751190027
AUC SCORE:  0.8556086432506886
######################### 模块循环 ： 667 ##########################
AP：  0.8619951612144262
AUC SCORE:  0.8553647268135904
AP：  0.8631706492182909
AUC SCORE:  0.8565089818640954
######################### 模块循环 ： 668 ##########################
AP：  0.8626437523691114
AUC SCORE:  0.8558669077134986
AP：  0.8636364428320955
AUC SCORE:  0.8567242022497704
######################### 模块循环 ： 669 ##########################
AP：  0.8624466382270881
AUC SCORE:  0.8557054924242424
AP：  0.86304686905131
AUC SCORE:  0.8564264807162536
######################### 模块循环 ： 670 ##########################
AP：  0.8627819172114867
AUC SCORE:  0.8557879935720846
AP：  0.8636287064052148
AUC SCORE:  0.8567098542240588
#######################

AP：  0.863197797160939
AUC SCORE:  0.8568856175390267
######################### 模块循环 ： 713 ##########################
AP：  0.8626814594414282
AUC SCORE:  0.8561287591827366
AP：  0.863032058066755
AUC SCORE:  0.856785181359045
######################### 模块循环 ： 714 ##########################
AP：  0.8626018779003247
AUC SCORE:  0.8559709308999082
AP：  0.863009816003557
AUC SCORE:  0.8566883321854913
######################### 模块循环 ： 715 ##########################
AP：  0.8625442187037264
AUC SCORE:  0.8558310376492195
AP：  0.8629480584829123
AUC SCORE:  0.8566022440312213
######################### 模块循环 ： 716 ##########################
AP：  0.8624800600524409
AUC SCORE:  0.8556732093663911
AP：  0.86297149720354
AUC SCORE:  0.8565161558769512
######################### 模块循环 ： 717 ##########################
AP：  0.8624861378870934
AUC SCORE:  0.8556301652892562
AP：  0.8629140973960536
AUC SCORE:  0.8563870236455464
######################### 模块循环 ： 718 ##########################
AP：  0.8623977443

AP：  0.8607736885897428
AUC SCORE:  0.8528323002754822
AP：  0.860896009610272
AUC SCORE:  0.8529901285583104
######################### 模块循环 ： 761 ##########################
AP：  0.8608954582484487
AUC SCORE:  0.8531192607897153
AP：  0.8613923106516485
AUC SCORE:  0.8537182908631771
######################### 模块循环 ： 762 ##########################
AP：  0.8605390088050467
AUC SCORE:  0.852918388429752
AP：  0.8613884446291615
AUC SCORE:  0.8539478592745637
######################### 模块循环 ： 763 ##########################
AP：  0.8603530188455688
AUC SCORE:  0.8526027318640955
AP：  0.8597108620232171
AUC SCORE:  0.8514728248393021
######################### 模块循环 ： 764 ##########################
AP：  0.8605418944337861
AUC SCORE:  0.8528753443526171
AP：  0.8607442098182759
AUC SCORE:  0.8532268709825528
######################### 模块循环 ： 765 ##########################
AP：  0.8603557979657801
AUC SCORE:  0.8527964302112029
AP：  0.8612816335604194
AUC SCORE:  0.8540554694674012
######################

In [11]:
embedding_dim = 8
n_clusters = 24
alpha = 1

ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
ini_node_embeddings = ini_embedding.weight.detach()

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(ini_node_embeddings)
cluster_centers = kmeans.cluster_centers_
cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)

raw_nodes_embedding = ini_node_embeddings     # t=0时刻的节点嵌入
raw_cluster_centers = cluster_centers     # t=0时刻的类簇质心
for module_epoch in range(800):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
    raw_cluster_centers = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)

######################### 模块循环 ： 0 ##########################
AP：  0.5418794067614294
AUC SCORE:  0.5256255739210285
AP：  0.5484676482705606
AUC SCORE:  0.5338900367309458
######################### 模块循环 ： 1 ##########################
AP：  0.5590601398347754
AUC SCORE:  0.5456554178145087
AP：  0.5627992402381653
AUC SCORE:  0.5480013200183655
######################### 模块循环 ： 2 ##########################
AP：  0.5716835112425671
AUC SCORE:  0.555534033516988
AP：  0.5731796266457008
AUC SCORE:  0.5557851239669421
######################### 模块循环 ： 3 ##########################
AP：  0.5802894527953864
AUC SCORE:  0.5579731978879706
AP：  0.5952699149241455
AUC SCORE:  0.582881370523416
######################### 模块循环 ： 4 ##########################
AP：  0.6010700889280756
AUC SCORE:  0.5874978477961432
AP：  0.6017904427872418
AUC SCORE:  0.5860271751606979
######################### 模块循环 ： 5 ##########################
AP：  0.604347830954562
AUC SCORE:  0.5874009986225897
AP：  0.6048429814158509
AU

AP：  0.794483110475842
AUC SCORE:  0.7393752869605142
######################### 模块循环 ： 48 ##########################
AP：  0.7955442940952062
AUC SCORE:  0.7403150826446281
AP：  0.7962047077723695
AUC SCORE:  0.740695305325987
######################### 模块循环 ： 49 ##########################
AP：  0.7976180353353819
AUC SCORE:  0.7422126090449954
AP：  0.7982426815486744
AUC SCORE:  0.7429013142791552
######################### 模块循环 ： 50 ##########################
AP：  0.7993013006688598
AUC SCORE:  0.743927198117539
AP：  0.7999448619270708
AUC SCORE:  0.7444903581267217
######################### 模块循环 ： 51 ##########################
AP：  0.8010720231311556
AUC SCORE:  0.745706353305785
AP：  0.8015064142967738
AUC SCORE:  0.7459538567493114
######################### 模块循环 ： 52 ##########################
AP：  0.8030446599601408
AUC SCORE:  0.7474890955004592
AP：  0.8035189704822544
AUC SCORE:  0.7479159492653811
######################### 模块循环 ： 53 ##########################
AP：  0.80460556772022

AP：  0.8459145751721828
AUC SCORE:  0.7986828512396693
AP：  0.8459490971045589
AUC SCORE:  0.7986469811753902
######################### 模块循环 ： 96 ##########################
AP：  0.8462045315827704
AUC SCORE:  0.7989052456382002
AP：  0.8463474773181294
AUC SCORE:  0.7990200298438934
######################### 模块循环 ： 97 ##########################
AP：  0.8468273924668478
AUC SCORE:  0.7995114497245179
AP：  0.8469248919540125
AUC SCORE:  0.7995544938016528
######################### 模块循环 ： 98 ##########################
AP：  0.8475085321209173
AUC SCORE:  0.8002396120293849
AP：  0.8477612302182269
AUC SCORE:  0.8005229855371901
######################### 模块循环 ： 99 ##########################
AP：  0.8480888373894825
AUC SCORE:  0.8007955980257115
AP：  0.8482416600517034
AUC SCORE:  0.8008852731864095
######################### 模块循环 ： 100 ##########################
AP：  0.8485744021739012
AUC SCORE:  0.8012906049127639
AP：  0.8486923684838311
AUC SCORE:  0.8013551710284663
########################

AP：  0.8651730036232467
AUC SCORE:  0.8247568009641872
AP：  0.8652987396684754
AUC SCORE:  0.8249146292470155
######################### 模块循环 ： 144 ##########################
AP：  0.8655772518286038
AUC SCORE:  0.8252661558769512
AP：  0.8657080572267637
AUC SCORE:  0.8253307219926538
######################### 模块循环 ： 145 ##########################
AP：  0.865998520598402
AUC SCORE:  0.8256499655647382
AP：  0.8662309836977149
AUC SCORE:  0.8259333390725436
######################### 模块循环 ： 146 ##########################
AP：  0.8665508492414407
AUC SCORE:  0.8263350837924701
AP：  0.8667406229761409
AUC SCORE:  0.8265395431588614
######################### 模块循环 ： 147 ##########################
AP：  0.8669586546733701
AUC SCORE:  0.8268121556473831
AP：  0.8671333245096031
AUC SCORE:  0.826944874885216
######################### 模块循环 ： 148 ##########################
AP：  0.8673360065689641
AUC SCORE:  0.8271923783287419
AP：  0.8675056322992492
AUC SCORE:  0.8274147727272727
######################

AP：  0.8797971518121068
AUC SCORE:  0.8460672061524335
######################### 模块循环 ： 191 ##########################
AP：  0.8799833006194615
AUC SCORE:  0.8464079717630855
AP：  0.8799974620512976
AUC SCORE:  0.8465335169880625
######################### 模块循环 ： 192 ##########################
AP：  0.8800824065509207
AUC SCORE:  0.8465980831037649
AP：  0.8801225801233071
AUC SCORE:  0.8467164543158862
######################### 模块循环 ： 193 ##########################
AP：  0.8802314210170754
AUC SCORE:  0.846845586547291
AP：  0.8803362993686236
AUC SCORE:  0.8469926538108357
######################### 模块循环 ： 194 ##########################
AP：  0.8804395821556985
AUC SCORE:  0.8472222222222222
AP：  0.8805130676023066
AUC SCORE:  0.8473477674471992
######################### 模块循环 ： 195 ##########################
AP：  0.880673875773039
AUC SCORE:  0.8475020087235998
AP：  0.8807390854435169
AUC SCORE:  0.8475952708907253
######################### 模块循环 ： 196 ##########################
AP：  0.880806

...

AP：  0.9022711591733137
AUC SCORE:  0.8845235020661157
######################### 模块循环 ： 571 ##########################
AP：  0.8997702461808763
AUC SCORE:  0.8811337809917357
AP：  0.9001870206432109
AUC SCORE:  0.8819946625344353
######################### 模块循环 ： 572 ##########################
AP：  0.8970966725197556
AUC SCORE:  0.8761729511019284
AP：  0.8994076976858947
AUC SCORE:  0.8809687786960514
######################### 模块循环 ： 573 ##########################
AP：  0.9001455057511202
AUC SCORE:  0.8808611685032139
AP：  0.9011541565261442
AUC SCORE:  0.8832967458677687
######################### 模块循环 ： 574 ##########################
AP：  0.9009938272458861
AUC SCORE:  0.8829165231864096
AP：  0.9008445361290063
AUC SCORE:  0.8833397899449036
######################### 模块循环 ： 575 ##########################
AP：  0.8973911000663447
AUC SCORE:  0.8772956841138659
AP：  0.8994334485096172
AUC SCORE:  0.8811624770431589
######################### 模块循环 ： 576 ##########################
AP：  0.9001

AP：  0.9011353503271097
AUC SCORE:  0.8831855486685032
AP：  0.9029636582513411
AUC SCORE:  0.8849395948117538
######################### 模块循环 ： 619 ##########################
AP：  0.8989581897098462
AUC SCORE:  0.8808037764003672
AP：  0.8998035746244157
AUC SCORE:  0.8815678087695134
######################### 模块循环 ： 620 ##########################
AP：  0.9024038962380744
AUC SCORE:  0.885316230486685
AP：  0.9027312559597075
AUC SCORE:  0.8839962121212122
######################### 模块循环 ： 621 ##########################
AP：  0.9019398337770165
AUC SCORE:  0.8841181703397611
AP：  0.9018602993598139
AUC SCORE:  0.883702077594123
######################### 模块循环 ： 622 ##########################
AP：  0.8924826762290036
AUC SCORE:  0.8724209423783287
AP：  0.8925113265110993
AUC SCORE:  0.8721698519283746
######################### 模块循环 ： 623 ##########################
AP：  0.8966763446860933
AUC SCORE:  0.8791071223599632
AP：  0.8987017302611238
AUC SCORE:  0.8815498737373737
######################

AP：  0.9012074716078494
AUC SCORE:  0.8827156508264463
######################### 模块循环 ： 666 ##########################
AP：  0.897791168745155
AUC SCORE:  0.8794048438934802
AP：  0.8938428386097335
AUC SCORE:  0.8743220557851239
######################### 模块循环 ： 667 ##########################
AP：  0.8977682940279804
AUC SCORE:  0.879627238292011
AP：  0.8977146682684937
AUC SCORE:  0.879379734848485
######################### 模块循环 ： 668 ##########################
AP：  0.8991559822502648
AUC SCORE:  0.8805670339761249
AP：  0.8998157299325037
AUC SCORE:  0.8825578225436179
######################### 模块循环 ： 669 ##########################
AP：  0.8986253172772447
AUC SCORE:  0.8794299529384757
AP：  0.8996773247792734
AUC SCORE:  0.8808073634067952
######################### 模块循环 ： 670 ##########################
AP：  0.9000428270522598
AUC SCORE:  0.8819408574380165
AP：  0.9006411238108994
AUC SCORE:  0.8824430383379247
######################### 模块循环 ： 671 ##########################
AP：  0.9003457

AP：  0.9006505266035653
AUC SCORE:  0.8823282541322314
AP：  0.9014407686095268
AUC SCORE:  0.8849647038567493
######################### 模块循环 ： 714 ##########################
AP：  0.9006398178779893
AUC SCORE:  0.8822744490358126
AP：  0.9015187499011346
AUC SCORE:  0.8854094926538107
######################### 模块循环 ： 715 ##########################
AP：  0.9006438214836603
AUC SCORE:  0.8823318411386594
AP：  0.9015485362858782
AUC SCORE:  0.8849216597796143
######################### 模块循环 ： 716 ##########################
AP：  0.9005122426095662
AUC SCORE:  0.8823390151515151
AP：  0.9012757585831135
AUC SCORE:  0.8849503558310375
######################### 模块循环 ： 717 ##########################
AP：  0.9005064678942363
AUC SCORE:  0.8822636880165289
AP：  0.9018200166352978
AUC SCORE:  0.8857717803030303
######################### 模块循环 ： 718 ##########################
AP：  0.9005848069031812
AUC SCORE:  0.88246814738292
AP：  0.9010186574034863
AUC SCORE:  0.8836913165748393
######################

AP：  0.900522001204642
AUC SCORE:  0.8848678546831956
######################### 模块循环 ： 761 ##########################
AP：  0.8991551447887505
AUC SCORE:  0.88110867194674
AP：  0.900111338870684
AUC SCORE:  0.883192722681359
######################### 模块循环 ： 762 ##########################
AP：  0.8992031529402817
AUC SCORE:  0.8811732380624426
AP：  0.899998886833359
AUC SCORE:  0.8834438131313131
######################### 模块循环 ： 763 ##########################
AP：  0.8990372116621528
AUC SCORE:  0.8809185606060606
AP：  0.9005984991110378
AUC SCORE:  0.8849539428374655
######################### 模块循环 ： 764 ##########################
AP：  0.8989902750815832
AUC SCORE:  0.8808898645546372
AP：  0.8997650046861646
AUC SCORE:  0.8825972796143251
######################### 模块循环 ： 765 ##########################
AP：  0.8994494405372873
AUC SCORE:  0.8814924816345271
AP：  0.9006272477294814
AUC SCORE:  0.8848463326446281
######################### 模块循环 ： 766 ##########################
AP：  0.8990429698